In [30]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime as dt
import datetime, os, wget, requests
import uuid
import xlwings as xw
from bs4 import BeautifulSoup

In [2]:
'vg_data/vendeeglobe_20210302_210000.xlsx'

'vg_data/vendeeglobe_20210302_210000.xlsx'

In [25]:
start_date = dt(2020, 11, 8)
end_date = dt(2021, 1, 26)
hours = ['14', '17', '21', '04', '08', '11']

In [26]:
start_date.strftime("%Y%m%d")

'20201108'

In [27]:
def download_excel(date, hour):
    str_date = date.strftime("%Y%m%d")
    url = f'https://www.vendeeglobe.org/download-race-data/vendeeglobe_{str_date}_{hour}0000.xlsx'
    try:
        if not os.path.exists(f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx'):
            wget.download(url, f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx')
    except Exception as e:
        print(f'Error for {date.strftime("%Y-%m-%d")} at {hour}H:')
        print(e)

In [28]:
date = start_date
while date <= end_date:
    str_date = date.strftime("%Y%m%d")
    for hour in hours:
        download_excel(date, hour)
    date += datetime.timedelta(days=1)
for hour in hours[:4]:
    download_excel(date, hour)

Error for 2020-11-08 at 04H:
HTTP Error 404: Not Found
Error for 2020-11-08 at 08H:
HTTP Error 404: Not Found


Faut il lire le premier ? Tous les compétiteurs sont au meme point de départ

In [20]:
def clean_excel(date, hour):
    str_date = date.strftime("%Y%m%d")
    data = None
    if os.path.exists(f'vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx'):
        with xw.Book(f"vg_data/vendeeglobe_{str_date}_{hour}0000.xlsx", mode="r") as book:
            sheet1 = book.sheets[0]
            data = sheet1.cells.options("df").value

        data = data.iloc[2:]
        data.iloc[1,:3] = data.iloc[0,:3]
        data.iloc[1,-2:] = data.iloc[0,-2:]
        data.iloc[1] = data.iloc[1].str.split("\r\n").str[0]
        data.iloc[1, 1] = data.iloc[1, 1].replace(" / ",' ')
        data.iloc[1, 2] = data.iloc[1, 2].replace(" / ",'\r\n')
        data.iloc[1,6:10] = data.iloc[1,6:10]+"_30m"
        data.iloc[1,10:14] = data.iloc[1,10:14]+"_last"
        data.iloc[1,14:18] = data.iloc[1,14:18]+"_24h"
        data.columns = data.iloc[1]
        data = data.iloc[2:]
        data = data.iloc[:-4]
        nat_voile = "(?P<Nationalite>.*) (?P<Voile>[0-9]*)"
        data[["Nationalité", "Voile"]] = data['Nat. Voile'].str.extract(nat_voile, expand=True)
        skipper_bateau = "(?P<Skipper>.*)\r\n(?P<nBateau>.*)"
        data[["Skipper", "Bateau"]] = data['Skipper\r\nBateau'].str.extract(skipper_bateau, expand=True)
        data = data.drop(['Nat. Voile', 'Skipper\r\nBateau'], axis=1)
        data["date"] = date 
        data = data.reset_index(drop=True)
    return data

In [70]:
date = start_date
race_df = pd.DataFrame()
while date <= end_date + datetime.timedelta(days=1):
    for hour in hours:
        tmp_df = clean_excel(date, hour)
        race_df = pd.concat([race_df, tmp_df])
    date += datetime.timedelta(days=1)
# race_df = race_df.reset_index(drop=True)
race_df

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [60]:
pd.date_range(start_date, end_date + datetime.timedelta(days=1))[0]

Timestamp('2020-11-08 00:00:00', freq='D')

In [67]:
date_premier_arrivé = dt(2021, 1, 27)

In [68]:
pd.concat([clean_excel(start_date, hours[1]), clean_excel(start_date, hours[0])]).reset_index(drop=True)

,Rang,Heure FR,Latitude,Longitude,Cap_30m,Vitesse_30m,VMG_30m,Distance_30m,Cap_last,Vitesse_last,...,Vitesse_24h,VMG_24h,Distance_24h,DTF,DTL,Nationalité,Voile,Skipper,Bateau,date
0,1,18:30 FR\r\n,46°19.75'N,03°27.02'W,268°,20.2 kts,16.8 kts,10.1 nm,268°,23.7 kts,...,3.0 kts,2.8 kts,69.5 nm,24234.9 nm,0.0 nm,FRA,8,Jérémie Beyou,Charal,2020-11-08
1,2,18:30 FR\r\n,46°15.92'N,03°21.99'W,266°,20.1 kts,17.3 kts,10.1 nm,267°,23.3 kts,...,2.8 kts,2.7 kts,66.8 nm,24235.6 nm,0.6 nm,FRA,79,Charlie Dalin,APIVIA,2020-11-08
2,3,18:30 FR\r\n,46°19.06'N,03°24.12'W,265°,20.9 kts,18.0 kts,10.4 nm,269°,23.9 kts,...,2.9 kts,2.7 kts,67.6 nm,24236.2 nm,1.2 nm,FRA,59,Thomas Ruyant,LinkedOut,2020-11-08
3,4,18:30 FR\r\n,46°15.33'N,03°18.94'W,265°,20.2 kts,17.6 kts,10.1 nm,264°,21.9 kts,...,2.8 kts,2.7 kts,64.9 nm,24237.0 nm,2.0 nm,GBR,99,Alex Thomson,Hugo Boss,2020-11-08
4,5,18:30 FR\r\n,46°17.04'N,03°18.71'W,269°,18.8 kts,15.7 kts,9.4 nm,268°,22.6 kts,...,2.7 kts,2.6 kts,64.3 nm,24238.1 nm,3.1 nm,MON,10,Boris Herrmann,Seaexplorer - Yacht Club De Monaco,2020-11-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,29,15:29 FR\r\n-1min,46°25.75'N,01°48.73'W,235°,13.4 kts,13.4 kts,0.7 nm,357°,0.0 kts,...,0.2 kts,0.1 kts,4.5 nm,24295.5 nm,1.6 nm,FRA,4,Sébastien Simon,ARKEA PAPREC,2020-11-08
62,30,15:28 FR\r\n-2min,46°25.39'N,01°48.34'W,237°,11.4 kts,11.3 kts,0.4 nm,358°,0.0 kts,...,0.2 kts,0.2 kts,4.8 nm,24295.6 nm,1.7 nm,FRA,50,Miranda Merron,Campagne de France,2020-11-08
63,31,15:30 FR\r\n,46°25.65'N,01°48.21'W,234°,12.1 kts,12.0 kts,0.2 nm,358°,0.0 kts,...,0.2 kts,0.2 kts,4.6 nm,24295.8 nm,1.9 nm,FIN,222,Ari Huusela,Stark,2020-11-08
64,NL,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,FRA,79,Charlie Dalin,APIVIA,2020-11-08


In [56]:
clean_excel(start_date, hours[0])["Voile"].unique()

array(['18', '10', '8', '59', '53', '56', '01', '99', '69', '777', '85',
       '92', '49', '1000', '83', '109', '17', '34', '11', '6', '71', '30',
       '7', '33', '09', '02', '72', '27', '4', '50', '222', '79', '14'],
      dtype=object)

In [ ]:
def extract_boat_infos(url):
    page = requests.get(url).content
    soup = BeautifulSoup(page)
    rows_info = [attr for attr in soup.find('dl', {'class', 'row'}).stripped_strings]
    beer_infos = {
        'Nom': soup.find('div', {'class', 'product-detail-info-title'}).find("h1").string,
        'Style': rows_info[1],
        'Contenu': int(rows_info[3][:3]),
        'Degré d’alcool': float(rows_info[5].strip()[:-1].replace(',', '.')),
        'Origine': rows_info[7],
        'Brasseur': rows_info[9]
    }
    return beer_infos

In [31]:
url = "https://www.vendeeglobe.org/fr/glossaire"
page = requests.get(url).content
soup = BeautifulSoup(page)

In [39]:
specs_ul = soup.find_all('ul', {"class","boats-list__popup-specs-list"})

In [46]:
specs_list = [elem.string.split(' : ') for elem in specs_ul[0].find_all('li')]
{elem[0]: elem[1] for elem in specs_list}

{'Numéro de voile': 'FRA 56',
 'Anciens noms du bateau': 'No Way Back, Vento di Sardegna',
 'Architecte': 'VPLP/Verdier',
 'Chantier': 'Persico Marine',
 'Date de lancement': '01 Août 2015',
 'Longueur': '18,28 m',
 'Largeur': '5,85 m',
 "Tirant d'eau": '4,50 m',
 'Déplacement (poids)': '7 t',
 'Nombre de dérives': 'foils',
 'Hauteur mât': '29 m',
 'Voile quille': 'monotype',
 'Surface de voiles au près': '320 m2',
 'Surface de voiles au portant': '570 m2'}

In [49]:
dict_list = []
for spec_ul in specs_ul:
    specs_list = [elem.string.split(' : ') for elem in spec_ul.find_all('li')]
    specs_dict = {elem[0]: elem[1] for elem in specs_list}
    dict_list.append(specs_dict)
specs_df = pd.DataFrame(dict_list)
specs_df

,Numéro de voile,Anciens noms du bateau,Architecte,Chantier,Date de lancement,Longueur,Largeur,Tirant d'eau,Déplacement (poids),Nombre de dérives,Hauteur mât,Voile quille,Surface de voiles au près,Surface de voiles au portant
0,FRA 56,"No Way Back, Vento di Sardegna",VPLP/Verdier,Persico Marine,01 Août 2015,"18,28 m","5,85 m","4,50 m",7 t,foils,29 m,monotype,320 m2,570 m2
1,FRA 49,"Gitana Eighty, Synerciel, Newrest-Matmut",Bruce Farr Design,Southern Ocean Marine (Nouvelle Zélande),08 Mars 2007,"18,28m","5,80m","4,50m",9t,2,28m,acier forgé,280 m2,560 m2
2,FRA72,"Famille Mary-Etamine du Lys, Initiatives Coeur...",Marc Lombard,MAG France,01 Mars 1998,"18,28m","5,54m","4,50m",9t,2,29 m,acier,260 m2,580 m2
3,17,Safran 2 - Des Voiles et Vous,Verdier - VPLP,CDK Technologies,12 Mars 2015,"18,28 m","5,80 m","4,50 m",8 t,foils,29 m,acier mécano soudé,310 m2,550 m2
4,08,NaN,VPLP,CDK Technologies,18 Août 2018,"18,28 m","5,85 m","4,50 m",8t,foils,29 m,acier,320 m2,600 m2
5,FRA 14,"Ecover3, Président, Gamesa, Kilcullen Voyager-...",Owen Clarke Design LLP - Clay Oliver,Hakes Marine - Mer Agitée,03 Août 2007,"18,28 m","5,65 m","4,50 m","7,9 tonnes",foils,29 m,basculante avec vérin,300 m²,610 m²
6,18,Banque Populaire VIII,Verdier - VPLP,CDK Technologies,09 Juin 2015,"18,28 m","5,80 m","4,50 m","7,6 t",foils,28 m,acier,300 m2,600 m2
7,ESP 33,Kingfisher - Educacion sin Fronteras - Forum M...,Owen Clarke Design,Martens Yachts,02 Février 2000,"18,28 m","5,30 m","4,50 m","8,9 t",2,26 m,acier,240 m2,470 m2
8,FRA 71,"Paprec-Virbac2, Estrella Damm, We are Water, L...",Bruce Farr Yacht Design,Southern Ocean Marine (Nouvelle-Zélande),02 Février 2007,"18,28 m","5,80 m","4,50 m",9 t,2 asymétriques,"28,50",basculante sur vérin hydraulique,270 m2,560 m2
9,FRA30,Macif - SMA,Verdier - VPLP,CDK - Mer Agitée,01 Mars 2011,"18,28 m","5,70 m","4,5 m","7,7 t",2,29 m,acier forgé,340 m2,570 m2
